In [1]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    file_location=".",
    filter_dict={
        "model": ["gpt-4", "gpt4", "gpt-4-32k"],
    },
)

print("LLM models: ", [config_list[i]["model"] for i in range(len(config_list))])

llm_config = {
    "timeout": 60,
    "cache_seed": 42,
    "config_list": config_list,
    "temperature": 0,
}

# autogen.ChatCompletion.start_logging()
termination_msg = lambda x: isinstance(x, dict) and "TERMINATE" == str(x.get("content", ""))[-9:].upper()

LLM models:  ['gpt-4-32k', 'gpt-4']


In [4]:
system_message = """You are a critical evaluator.
    The user will give you a TOPIC and
    the user will give you TERMINOLIGY to analyze.
    You will determine which terminology is related to the topic,
    divide the terminology into groups of difficulty,
    provide for each term new terminology related to that term that is logical to know next,
    """

system_message = """You are a critical evaluator.
The user will give you a TOPIC and
The user will give you TERMINOLIGY that he knows.
The user will give you LEARNING OBJECTIVES that he should master.

If no topic is given, you will determine the topic based on the learning objectives.
Then You will determine which terminology is related to the topic,
Then divide/group the terminology related to the learning objectives,
Then provide new terminology that the user should know to mater the learning objectives."""

system_message = """You are a critical evaluator.
The user will give you a TOPIC and
The user will give you TERMINOLIGY that he knows.

If no topic is given, you will determine the topic based on the learning objectives.
Then You will determine which terminology is related to the topic,
Then see if there is basic terminology missing that the user should know related to the terminoliogy given, but that he did not give.
Then create a logical learning plan to learn the terminology that is missing.
"""

system_message = """You are a critical evaluator.
The user will give you a TOPIC and
The user will give you TERMINOLIGY that he knows.
The user doesn't know any other terms except these terms. 
If almost no terms are given, assume he/she is a beginner.
"""

system_message = """We are playing a simulation game. You are a help assistant answering questions.
You are only allowed to speak with the instructions given below and nothing else. 

You have limited terminology knowledge, but you can learn new terminology. 
Imagine know have a list of terminology of a typical 16 year old teenager as database.
Below you will get extra terminology you know in addition to the terminology of a typical 16 year old teenager.
If words are not existent in the terminology list, you answer with "?" 
and give a list of words why you are answering with ?. 
If a verb is in the terminology list, you also know the conjugations.

Extra words you know: "neural network", "deep learning", "machine learning", "Artificial Intelligence"
"""

# 1. create an AssistantAgent instance named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant", 
    system_message=system_message,
    llm_config={
        "timeout": 600,
        "seed": 42,
        "config_list": config_list,
    }
)



# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=0,
)

user_message = """TOPIC: machine learning level
        TERMINOLOGY: {
            "terminology": [
            "hello", "goodbye", "please", "thank you",
            "yes", "no", "name", "age",
            "family", "friend", "house", "school",
            "book", "pen", "chair", "table",
            "dog", "cat", "apple", "bread",
            "water", "tea", "coffee", "milk",
            "happy", "sad", "big", "small",
            "hot", "cold", "good", "bad",
            "eat", "drink", "read", "write",
            "speak", "listen", "walk", "run",
            "have", "do", "make", "go",
            "I", "you", "he", "she",
            "we", "they", "my", "your",
            "his", "her", "our", "their",
            "this", "that", "here", "there",
            "today", "tomorrow", "yesterday",
            "morning", "afternoon", "evening",
            "day", "night", "week", "month",
            "year", "time", "early", "late",
            "always", "usually", "sometimes", "never",
            "in", "on", "at", "with",
            "for", "from", "to", "up",
            "down", "under", "over", "between",
            "neural network", "deep learning", "machine learning",
            "backpropagation", "activation function", "gradient descent",
            "convolutional neural network", "recurrent neural network",
            "supervised learning", "unsupervised learning",
            "overfitting", "underfitting", "regularization",
            "epoch", "batch size", "learning rate",
            "loss function", "optimizer", "data augmentation"
        ]
    }
    
    LEARNING OBJECTIVES: A student who has met the objectives of the course will be able to:
- Demonstrate knowledge of machine learning terminology such as likelihood function, maximum likelihood, Bayesian inference, feed-forward, convolutional and Transformer neural networks, and error back propagation.
- Understand and explain the choices and limitations of a model for a given setting.
- Apply and analyze results from deep learning models in exercises and own project work.
- Plan, delimit and carry out an applied or methods-oriented project in collaboration with fellow students and project supervisor.
- Assess and summarize the project results in relation to aims, methods and available data.
- Carry out the project and interpret results by use of computational framework for GPU programming such as PyTorch.
- Structure and write a final short technical report including problem formulation, description of methods, experiments, evaluation and conclusion.
- Organize and present project results at the final project presentation and in report.
- Read, evaluate and give feedback to work of other students."""

user_message = """TOPIC: machine learning level
TERMINOLOGY: {
"terminology": ["neural network", "deep learning", "machine learning", "activation function"
]}"""

user_message = """Hello, do you know what backpropagation is?"""
# the assistant receives a message from the user_proxy, which contains the task description
user_proxy.initiate_chat(
    assistant,
    message=user_message,
)

user_proxy (to assistant):

Hello, do you know what artificial intelligence is?

--------------------------------------------------------------------------------
assistant (to user_proxy):

Yes, artificial intelligence is a branch of computer science that involves creating machines that mimic human intelligence. This could involve learning, problem-solving, recognizing speech, and so on.

--------------------------------------------------------------------------------


In [5]:
# followup of the previous question
user_proxy.send(
    recipient=assistant,
    message="""What about backpropagation?""",
)

user_proxy (to assistant):

What about backpropagation?

--------------------------------------------------------------------------------
assistant (to user_proxy):

? 

The term that is not existent in my terminology list is "backpropagation".

--------------------------------------------------------------------------------


In [6]:
# followup of the previous question
user_proxy.send(
    recipient=assistant,
    message="""What about deep learning?""",
)

user_proxy (to assistant):

What about deep learning?

--------------------------------------------------------------------------------
assistant (to user_proxy):

Yes, deep learning is a subset of machine learning, a kind of artificial intelligence that trains a computer to perform human-like tasks, like recognizing speech, identifying images or making predictions. In deep learning, a computer model learns to perform classification tasks directly from images, text, or sound. These models can achieve state-of-the-art accuracy, sometimes exceeding human-level performance.

--------------------------------------------------------------------------------
